# Construction d'un système de recommandation

Nous avons décidé d'orienter notre projet sur la recommendation de films.
En effet durant ce confinement, nous avons eu le temps de visionner beaucoup de films,
mais nous nous sommes rendus compte que nous passions quasiment autant de temps
à choisir le film qu'à le regarder. D'où la nécessité de créer un système de re-
commendations afin d'optimiser notre temps de visionnage.
Nous avons chercher une base de données assez exploitable afin de mener à bien
notre projet. Nous nous sommes basés sur la base de données de 'The Movies Dataset'.


In [135]:
import numpy as np
import pandas as pd
import math
import re

## Fetching and cleaning data

Nous utilisons deux tables de données. L'une, *movies_metadata.csv*, contient une liste de films et des informations relativesau genre, date de sortie etc. 

### Informations sur les films

In [136]:
movies = pd.read_csv("movies_metadata.csv")
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [12]:
movies.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [13]:
def filter_correct_id(word):
    if re.fullmatch(r'[0-9]+', word):
        return word
    return "wrong_id"

In [14]:
# don't re-run
movies = movies[~movies.id.duplicated()]
movies.id = movies.id.apply(filter_correct_id)
movies = movies[movies.id != "wrong_id"]
movies.id = movies.id.astype('int64')

### Avis des utilisateurs

In [15]:
ratings = pd.read_csv("ratings_small.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [16]:
# ne pas re-run !
ratings = ratings.drop(columns=['timestamp'])
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [17]:
ratings[(ratings['userId'] == 1) & (ratings['movieId'] == 31)]

,userId,movieId,rating
0,1,31,2.5


In [18]:
print(min(ratings.rating), max(ratings.rating))
ratings.describe()
ratings.dtypes

0.5 5.0


userId       int64
movieId      int64
rating     float64
dtype: object

In [19]:
nbPers = len(ratings.userId.unique())
nbMovi = len(ratings.movieId.unique())

## User Based Recommandation

Pour ce système, nous n'avons que besoin des notations des utilisateurs et des titres des films associés. Nous allons translater les notes afin que la moyenne des notes pour chaque utilisateur se trouve à 0. Par abus de langage nous appelons ces nouvelles notes les notes *normalisées*. 

In [168]:
movies_title = movies.loc[movies['id'].isin(ratings.movieId), ['id', 'title']]
print(len(ratings.movieId.unique()))
ratings_small = ratings.loc[ratings['userId'] <= 50]

9066


In [181]:
def mean_user(df, uid):
    '''
    Retourne la moyenne des notes données par l'utilisateur d'id uid
    '''
    n = df.loc[ratings['userId'] == uid].count().loc['userId']
    s = df.loc[ratings['userId'] == uid].sum().loc['rating']
    return s / n

In [182]:
def normalize_user(df):
    '''
    Ajoute une colonne dans la dataframe df contenant les notes normalisées des utilisateurs
    '''
    mean = df.loc[:, ['userId']].drop_duplicates()
    mean['mu'] = mean['userId'].map(lambda uid : mean_user(df, uid))
    mean = mean.set_index('userId')
    df['rating_norm'] = df[['userId', 'rating']].apply(lambda row : row['rating'] -  mean.loc[int(row['userId'])]['mu'], axis=1)

In [183]:
normalize_user(ratings_small)
print(ratings_small.head())

   userId  movieId  rating  rating_norm
0       1       31     2.5        -0.05
1       1     1029     3.0         0.45
2       1     1061     3.0         0.45
3       1     1129     2.0        -0.55
4       1     1172     4.0         1.45


/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Regrouper les utilisateurs dans des peer-group

Pour déterminer si deux utilisateurs se ressemblent en termes de goûts, nous utilisons un taux de corrélation sur les avis données. Nous allons comparer quatres taux de corrélations différents. Le premier ```cor()``` calcule le taux de corrélation classique donné par la formule :
$$
cor(u, v) = \frac{\sum_{k \in I_{uv}} s_{uk} s_{vk}}{\sqrt{\sum_{k \in I_{uv}} s_{uk}^2}\sqrt{\sum_{k \in I_{uv}} s_{vk}^2}}
$$

Le taux de corrélation ajusté ```cor_adj()``` permet de ne pas donner trop d'importance aux films populaires que beaucoup de personnes ont vu.

Le taux de correlation calculé par ```cor_dis()``` permet de ne pas donner une correlation trop élevée si les deux utilisateurs n'ont pas donné assez d'avis sur des films en commun. 

Enfin la fonction ```cor_dis_adj()``` fait un mélange des deux dernières amélioration : il filtre les films trop populaire et n'apporte de l'importance seulement si deux personnes ont données leur avis sur un certain nombre de films.

In [24]:
def cor(u, v, df, Iuv):
    su = df.loc[(df['userId'] == u) & (df['movieId'].isin(Iuv['movieId']))].rating_norm
    sv = df.loc[(df['userId'] == v) & (df['movieId'].isin(Iuv['movieId']))].rating_norm
    su = np.array(su)
    sv = np.array(sv)
    
    return np.dot(su, sv) / math.sqrt(np.dot(su, su) * np.dot(sv, sv))

In [25]:
def cor_adj(u, v, df, Iuv):
    nb_rat = df.loc[:, ['movieId', 'rating']].groupby(['movieId']).count()
    
    sum_up = 0
    sum_down_u = 0
    sum_down_v = 0
    for movie in Iuv.movieId.unique() :
        suk = df.loc[(df['userId'] == u) & (df['movieId'] == movie), ['rating_norm']]
        svk = df.loc[(df['userId'] == v) & (df['movieId'] == movie), ['rating_norm']]
        suk, svk = float(suk), float(svk)
        
        sum_up += suk * svk / nb_rat.at[movie, 'rating']
        sum_down_u += suk**2 /  nb_rat.at[movie, 'rating']
        sum_down_v += svk**2 /  nb_rat.at[movie, 'rating']
    return sum_up / math.sqrt(sum_down_u * sum_down_v)

In [26]:
def cor_dis(u, v, df, Iuv):
    beta = 20
    correlation = cor(u, v, df, Iuv)
    return correlation * min(len(Iuv), beta)/beta

In [27]:
def cor_dis_adj(u, v, df, Iuv):
    beta = 20
    correlation = cor_adj(u, v, df, Iuv)
    return correlation * min(len(Iuv), beta)/beta

**Test des taux de corrélation sur les utilisateurs 2 et 3 qui ont 8 films en communs**

In [28]:
u, v = 2, 3
df = ratings_small
Iu = df.loc[df['userId'] == u, ['movieId']]
Iv = df.loc[df['userId'] == v, ['movieId']]
Iuv = Iu.join(Iv.set_index('movieId'), on='movieId', how='inner')
print(Iuv)

print(cor(u, v, df, Iuv))
# print(cor_adj(u, v, df))
print(cor_dis(u, v, df, Iuv))
# print(cor_dis_adj(u, v, df))

    movieId
27      110
49      296
57      356
64      377
79      527
88      588
91      592
92      593
-0.016060945830838957
-0.006424378332335582


Nous construisons maintenant la matrice de correlation. Puisqu'une telle matrice est symétrique, nous avons préféré utiliser une dataframe à deux entrées et ne stocker la corrélation pour un couple qu'une seule fois.

In [53]:
def uu_matrix(df, cor_fct=cor):
    '''
    Retourne la dataframe des taux de corrélations des utilisateurs de df
    '''
    correlation = []
    users = df.userId.unique()
    couples = []
    for i in range(len(users)):
        u = users[i]
        if not u % 20 : 
            print('user:', u, end='')
        for j in range(i + 1, len(users)):
            v = users[j]
            Iu = df.loc[df['userId'] == u, ['movieId']]
            Iv = df.loc[df['userId'] == v, ['movieId']]
            Iuv = Iu.join(Iv.set_index('movieId'), on='movieId', how='inner')
            if Iuv.size :
                couples.append((u, v))
                correlation.append(cor_fct(u, v, df, Iuv))
    index = pd.MultiIndex.from_tuples(couples, names=['u', 'v'])
    cor = pd.DataFrame(correlation, index=index, columns=['correlation'])
    return cor

In [54]:
def get_cor(u, v, cm):
    '''
    Retourne le taux de correlation entre u et v stocké dans cm
    '''
    if u > v :
        u, v = v, u
    index = list(cm.index.values)
    if (u, v) in index :
        return float(cm.loc[(u, v)])
    return float('nan')

In [55]:
cm_small = uu_matrix(ratings_small)

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


user: 20user: 40

In [56]:
cm_small.head()

correlation
u v              
1 4      0.042137
  5     -1.000000
  7     -0.752427
  9      1.000000
  15     0.043773

In [57]:
print(get_cor(1, 4, cm_small))
print(get_cor(4, 1, cm_small))
print(get_cor(2, 1, cm_small))

0.042136808375910856
0.042136808375910856
nan


Nous allons maintenant prédire la note qu'un utilisateur **u** donnerait à un film *m . Pour cela, nous allons faire la somme des notes données à l'item *i* par les k utilisateurs plus proches de u qui ont donné une note à m. Cette somme sera pondérée par les coéfficients de corrélations.

$$
\hat{\sigma}_{um} = \mu_u + \frac{\sum_{v \in P_u(m)} s_{vm} \cdot cor(u, v)}{\sum_{v \in P_u(m)} |cor(u,v)|}
$$

Le peer-group de l'utilisateur u pour le film m est l'ensemble des k utilisateurs qui ont donné une note au film m les plus proche de l'utilisateur u en terme de taux de corrélation. 

In [104]:
def peers(user, movie, k, cm, df):
    '''
    Retourne les k utilisateurs du peer-group de (user, movie)
    '''
    top = [(float('-inf'), user)] * k
    df_movie = df.loc[df['movieId'] == movie, ['userId', 'rating_norm']]
    for v in df_movie.userId.unique():
        taux = get_cor(user, v, cm)
        if taux > top[-1][0] :
            top += [(taux, v)]
            top.sort(reverse=True)
            top = top[:-1]
    return [t[1] for t in top]

In [105]:
def predict(user, movie, k, cm, df):
    mu = mean_user(user)
    peer_group = peers(user, movie, k, cm, df)
    sum_up, sum_down = 0, 0
    for v in peer_group:
        cor = get_cor(user, v, cm)
        if not math.isnan(cor):
            svm = df.loc[(df['userId'] == v) & (df['movieId'] == movie), 'rating_norm']
            svm = 0 if len(svm) <= 0 else float(svm)
            sum_up += svm * cor
            sum_down += abs(cor)
    sum_down = 1 if sum_down == 0 else sum_down
    return mu + sum_up / sum_down

In [106]:
user = 85
movie = 10
k = 4

friends = peers(user, movie, k, cm_small, ratings_small)
p = predict(user, movie, k, cm_small, ratings_small)
print(friends)
print(p)
print(df.loc[(df['userId'].isin(friends)) & (df['movieId'] == 10)])

[19, 2, 7, 21]
2.9266428360943952
      userId  movieId  rating  norm_movie
3112      19       10     3.0    -0.45082
3626      21       10     3.0    -0.45082
20         2       10     4.0     0.54918
496        7       10     3.0    -0.45082


## Item Based Recommandation

Nous allons maintenant construire un système de recommandation basé sur les items. Dans le système user-based, pour prédire la note donnée par l'utilisateur *user* au film *movie*, on regardait les notes donnés à ce même film *movie* par des utilisateurs similaires à *user*. De manière analogue, dans le système item-based, pour prédire la note donnée par user à *movie*, on regarde les notes données par *user* à des films similaires à *movie*.

In [107]:
ratings_mm = ratings.loc[ratings['movieId'] <= 100]

**Normalisation des notes données aux films par leur moyenne**

In [185]:
def mean_movie(df, mid):
    '''
    Retourne la moyenne des notes données au film d'id mid
    '''
    n = df.loc[ratings['movieId'] == mid].count().loc['movieId']
    s = df.loc[ratings['movieId'] == mid].sum().loc['rating']
    return s / n

In [186]:
def normalize_movie(df):
    '''
    Ajoute une colonne dans la dataframe df contenant les notes normalisées des films
    '''
    mean = df.loc[:, ['movieId']].drop_duplicates()
    mean['mu'] = mean['movieId'].map(lambda mid : mean_movie(df, mid))
    mean = mean.set_index('movieId')
    df['norm_movie'] = df[['movieId', 'rating']].apply(lambda row : row['rating'] -  mean.loc[int(row['movieId'])]['mu'], axis=1)

In [187]:
normalize_movie(ratings_mm)

In [111]:
ratings_mm = ratings_mm.sort_values(by='movieId')
ratings_mm.head()

,userId,movieId,rating,norm_movie
44632,313,1,4.0,0.12753
30672,219,1,5.0,1.12753
92559,615,1,4.0,0.12753
92944,616,1,4.0,0.12753
59530,431,1,4.5,0.62753


In [112]:
def cor_movie(u, v, df, Iuv):
    su = df.loc[(df['movieId'] == u) & (df['userId'].isin(Iuv['userId']))].norm_movie
    sv = df.loc[(df['movieId'] == v) & (df['userId'].isin(Iuv['userId']))].norm_movie
    su = np.array(su)
    sv = np.array(sv)
    
    return np.dot(su, sv) / math.sqrt(np.dot(su, su) * np.dot(sv, sv))

In [119]:
def mm_matrix(df, cor_fct=cor_movie):
    '''
    Retourne la dataframe des taux de corrélations des utilisateurs de df
    '''
    correlation = []
    movies = df.movieId.unique()
    couples = []
    for i in range(len(movies)):
        u = movies[i]
        if not i % 20 : 
            print('movie number:', i, end=' ')
        for j in range(i + 1, len(movies)):
            v = movies[j]
            Iu = df.loc[df['movieId'] == u, ['userId']]
            Iv = df.loc[df['movieId'] == v, ['userId']]
            Iuv = Iu.join(Iv.set_index('userId'), on='userId', how='inner')
            if Iuv.size :
                couples.append((u, v))
                correlation.append(cor_fct(u, v, df, Iuv))
    index = pd.MultiIndex.from_tuples(couples, names=['u', 'v'])
    cor = pd.DataFrame(correlation, index=index, columns=['correlation'])
    return cor

In [114]:
cm_small = mm_matrix(ratings_mm)

movie number:  0 

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


movie number:  1 movie number:  2 movie number:  3 movie number:  4 movie number:  5 movie number:  6 movie number:  7 movie number:  8 movie number:  9 movie number:  10 movie number:  11 movie number:  12 movie number:  13 movie number:  14 movie number:  15 movie number:  16 movie number:  17 movie number:  18 movie number:  19 movie number:  20 movie number:  21 movie number:  22 movie number:  23 movie number:  24 movie number:  25 movie number:  26 movie number:  27 movie number:  28 movie number:  29 movie number:  30 movie number:  31 movie number:  32 movie number:  33 movie number:  34 movie number:  35 movie number:  36 movie number:  37 movie number:  38 movie number:  39 movie number:  40 movie number:  41 movie number:  42 movie number:  43 movie number:  44 movie number:  45 movie number:  46 movie number:  47 movie number:  48 movie number:  49 movie number:  50 movie number:  51 movie number:  52 movie number:  53 movie number:  54 movie number:  55 movie number:  56 m

In [115]:
cm_small.head()

correlation
u v             
1 2     0.045297
  3     0.153406
  4     0.358796
  5    -0.490730
  6    -0.096954

In [116]:
def peers_movie(user, movie, k, cm, df):
    '''
    Retourne les k films du peer-group de (movie, user)
    '''
    top = [(float('-inf'), movie)] * k
    df_user = df.loc[df['userId'] == user, ['movieId', 'norm_movie']]
    for v in df_user.movieId.unique():
        taux = get_cor(movie, v, cm)
        if taux > top[-1][0] :
            top += [(taux, v)]
            top.sort(reverse=True)
            top = top[:-1]
    return [t[1] for t in top]

In [117]:
def predict(user, movie, k, cm, df):
    mu = mean_movie(movie)
    sum_up, sum_down = 0, 0
    peer_group = peers_movie(user, movie, k, cm, df)
    for v in peer_group:
        cor = get_cor(movie, v, cm)
        if not math.isnan(cor):
            svm = df.loc[(df['movieId'] == v) & (df['userId'] == user), 'norm_movie']
            svm = 0 if len(svm) <= 0 else float(svm)
            sum_up += svm * cor
            sum_down += abs(cor)
    sum_down = 1 if sum_down == 0 else sum_down
    return mu + sum_up / sum_down

In [118]:
cm = cm_small
df = ratings_mm
user = 85
movie = 10
k = 4

friends = peers_movie(user, movie, k, cm, df)
p = predict(user, movie, k, cm, df)
print(friends)
print(p)
print(df.loc[(df['movieId'].isin(friends)) & (df['userId'] == user)])

[21, 5, 2, 44]
3.742214146340207
       userId  movieId  rating  norm_movie
13184      85        2     5.0    1.598131
13186      85        5     3.0   -0.267857
13189      85       21     4.0    0.463158
13191      85       44     2.0   -0.697368


## Factorisation du user- et item- based

Dans un système user-based, le ptype (pour peer-type) est userId et le otype (pour other-type) est movieId. Dans un système item-based c'est l'inverse. 

In [422]:
ratings_small = ratings.loc[(ratings['userId'] <= 100) & (ratings['movieId'] <= 100)]
ratings_small.head()

,userId,movieId,rating
0,1,31,2.5
20,2,10,4.0
21,2,17,5.0
22,2,39,5.0
23,2,47,4.0


### Normalisation

In [423]:
def mean_base(df, pid, base):
    '''
    Retourne la moyenne des notes données par l'utilisateur d'id uid
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    n = df.loc[ratings[ptype] == pid].count().loc[ptype]
    s = df.loc[ratings[ptype] == pid].sum().loc['rating']
    return s / n

In [424]:
pid = 4
print(mean_base(ratings_small, pid, 'movie'))
print(mean_base(ratings_small, pid, 'user'))

3.0
4.5


In [425]:
def normalize(df, base='user'):
    '''
    Ajoute une colonne dans la dataframe df contenant les notes normalisées des utilisateurs
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    mean = df.loc[:, [ptype]].drop_duplicates()
    mean['mu'] = mean[ptype].map(lambda pid : mean_base(df, pid, base))
    mean = mean.set_index(ptype)
    df['rating_norm'] = df[[ptype, 'rating']].apply(lambda row : row['rating'] -  mean.loc[int(row[ptype])]['mu'], axis=1)
    df.sort_values(by=ptype, inplace=True)

In [426]:
normalize(ratings_small, 'movie')

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [427]:
ratings_small.head()

,userId,movieId,rating,rating_norm
12616,79,1,2.0,-1.828947
889,13,1,5.0,1.171053
962,15,1,2.0,-1.828947
9610,67,1,3.0,-0.828947
13291,86,1,3.0,-0.828947


### Matrice de correlation

# Big question : on donne quoi comme correlation si su et/ou sv est nul ? J'ai mis 0 par défaut mais bon ...

In [428]:
def cor(df, u, v, Iuv, base):
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    su = df.loc[(df[ptype] == u) & (df[otype].isin(Iuv[otype]))].rating_norm
    sv = df.loc[(df[ptype] == v) & (df[otype].isin(Iuv[otype]))].rating_norm
    su = np.array(su)
    sv = np.array(sv)
    up = np.dot(su, sv)
    down = math.sqrt(np.dot(su, su) * np.dot(sv, sv))
    if up == 0 or down == 0:
        return 0
    return up/ down

In [429]:
def cor_matrix(df, cor_fct=cor, base='user'):
    '''
    Retourne la dataframe des taux de corrélations des ptype de df
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    correlation = []
    peers = df.userId.unique() if base == 'user' else df.movieId.unique()
    couples = []
    for i in range(len(peers)):
        u = peers[i]
        if not u % 20 : 
            print('peer :', u, end='\n')
        for j in range(i + 1, len(peers)):
            v = peers[j]
            Iu = df.loc[df[ptype] == u, [otype]]
            Iv = df.loc[df[ptype] == v, [otype]]
            Iuv = Iu.join(Iv.set_index(otype), on=otype, how='inner')
            if Iuv.size :
                couples.append((u, v))
                correlation.append(cor_fct(df, u, v, Iuv, base))
    index = pd.MultiIndex.from_tuples(couples, names=['u', 'v'])
    cor = pd.DataFrame(correlation, index=index, columns=['correlation'])
    return cor

In [430]:
cm_small = cor_matrix(ratings_small, base='movie')

peer : 20
peer : 40
peer : 60
peer : 100


In [431]:
cm_small.head()

correlation
u v             
1 2     0.212100
  3     0.535777
  4     0.000000
  5     0.177455
  6     0.193060

### Prédire une note pour un couple p/o donné

In [432]:
def peers(df, cm, k, p, o, base='user'):
    '''
    Retourne les k films du peer-group de (movie, user)
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    top = [(float('-inf'), p)] * k
    df_peers = df.loc[df[ptype] == p, [otype, 'rating_norm']]
    peers = df_peers.iloc[0].unique()
    for v in peers:
        taux = get_cor(p, v, cm)
        if taux > top[-1][0] :
            top += [(taux, v)]
            top.sort(reverse=True)
            top = top[:-1]
    return [t[1] for t in top]

In [433]:
def predict(df, cm, k, p, o, base='user'):
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    mu = mean_base(df, p, base)
    peer_group = peers(df, cm, k, p, o, base)
    sum_up, sum_down = 0, 0
    for v in peer_group:
        cor = get_cor(p, v, cm)
        if not math.isnan(cor):
            svm = df.loc[(df[ptype] == v) & (df[otype] == o), 'rating_norm']
            svm = 0 if len(svm) <= 0 else float(svm)
            sum_up += svm * cor
            sum_down += abs(cor)
    sum_down = 1 if sum_down == 0 else sum_down
    return mu + sum_up / sum_down

In [434]:
df = ratings_small
cm = cm_small
user = 85
movie = 10
k = 4

friends = peers(df, cm, k, user, movie, base='user')
p = predict(df, cm , k, user, movie, base='user')
print(friends)
print(p)
print(df.loc[(df['movieId'].isin(friends)) & (df['userId'] == user)])

[85, 85, 85, 85]
3.111111111111111
Empty DataFrame
Columns: [userId, movieId, rating, rating_norm]
Index: []
